# Data processing

## Background

The fake vs real news dataset has been downloaded from [Kaggle](https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset). This data set includes a data for 20,826 articles considered to be 'real' news and 17,903 articles considered to be fake news.

The stated task is: Can you use this data set to make an algorithm able to determine if an article is fake news or not ?

## Import packages

In [1]:
import pandas as pd

## Read in datasets

In [7]:
real_df = pd.read_csv('~/documents/Data/Fake vs Real News/True.csv')
fake_df = pd.read_csv('~/documents/Data/Fake vs Real News/Fake.csv')

In [14]:
print('real_df.dtypes)
print(fake_df.dtypes)

title      object
text       object
subject    object
date       object
dtype: object
title      object
text       object
subject    object
date       object
dtype: object
